# Behavioural cloning project

In [1]:
# Configure matlab to show graphics in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Change to 'data' when training on a GPU
PATH_TO_DATA = 'data/sample'

In [3]:
# Shape of the image
input_shape = (66, 200, 3)

In [4]:
def img_folder(data_folder):
    return '{}/IMG'.format(data_folder)

def path_driving_log(data_folder):
    return '{}/driving_log.csv'.format(data_folder)

In [5]:
datasets = ['train', 'test', 'valid']

In [6]:
def get_driving_log_dataframe(data_folder):
    driving_log_df = pd.read_csv(path_driving_log(data_folder))
    return driving_log_df

In [7]:
path_to_folders = dict(zip(datasets, map(lambda folder: '{0}/{1}'.format(PATH_TO_DATA, folder), datasets)))

In [8]:
path_to_folders

{'test': 'data/sample/test',
 'train': 'data/sample/train',
 'valid': 'data/sample/valid'}

In [9]:
!pip install imutils

In [10]:
import imutils

In [25]:
from scipy.ndimage import imread
from os import listdir
from sklearn.utils import shuffle

from PIL import Image

def image_label_generator(data_folder, batch_size=64):
    driving_log_df = get_driving_log_dataframe(data_folder)
    number_of_examples = len(driving_log_df)
    image_columns = ['center', 'left', 'right']
    
    X_train = []
    y_train = []
    weights = []
    index_in_batch = 0
    batch_number = 0
    
    while True:
        for image_column in image_columns:
            image_series = driving_log_df[image_column]
            steering_series = driving_log_df['steering']
            for offset in range(0, number_of_examples, batch_size):
                X_train = []
                y_train = []
                weights = []

                end_of_batch = min(number_of_examples, offset + batch_size)

                for j in range(offset, end_of_batch):
                    image_filename = image_series[j].lstrip().rstrip()
                    image = Image.open('{0}/{1}'.format(data_folder, image_filename))
                    image = np.asarray(image.resize((200, 66)))
                    label = steering_series[j]
                    
                    if abs(label) < 10e-4:
                        weights.append(1)
                    else:
                        weights.append(10)
                    
                    X_train.append(image)
                    y_train.append(label)
                    X_train, y_train, weights = shuffle(X_train, y_train, weights)

                yield np.array(X_train), np.array(y_train), np.array(weights)

In [12]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import Flatten

Using TensorFlow backend.


In [13]:
model = Sequential()

In [14]:
model.add(Convolution2D(3, 1, 1, 
                          input_shape=input_shape, 
                          border_mode='same', 
                          activation='relu',
                          init='he_normal'))
model.add(BatchNormalization())
model.add(Convolution2D(24, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Convolution2D(36, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Convolution2D(48, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Flatten())
model.add(Dense(100, activation='relu', init='he_normal'))
model.add(Dense(50, activation='relu', init='he_normal'))
model.add(Dense(10, activation='relu', init='he_normal'))
model.add(Dense(1, activation='linear', init='he_normal'))

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 66, 200, 3)    12          convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 66, 200, 3)    12          convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 31, 98, 24)    1824        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 14, 47, 36)    21636       convolution2d_2[0][0]            
___________________________________________________________________________________________

In [16]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

In [17]:
from keras.optimizers import Adam
adam = Adam(lr=10e-4)

In [18]:
model.compile(optimizer=adam, 
              loss='mse',
             metrics=['mean_squared_error'])

In [26]:
image_generator = image_label_generator(PATH_TO_DATA)
validation_generator = image_label_generator(path_to_folders['valid'])
samples = len(get_driving_log_dataframe(PATH_TO_DATA))
nb_val_samples = len(get_driving_log_dataframe(path_to_folders['valid']))

In [27]:
model.fit_generator(image_generator, 
                    samples_per_epoch=samples, 
                    nb_epoch=10,
                   callbacks=callbacks_list)

Epoch 1/10
19/19 [==============================] - 0s - loss: 5.5258 - mean_squared_error: 0.6412
Epoch 2/10
19/19 [==============================] - 0s - loss: 4.6247 - mean_squared_error: 0.8633
Epoch 3/10
19/19 [==============================] - 0s - loss: 22.0791 - mean_squared_error: 2.6391
Epoch 4/10
19/19 [==============================] - 0s - loss: 21.7424 - mean_squared_error: 2.5774
Epoch 5/10
19/19 [==============================] - 0s - loss: 21.5573 - mean_squared_error: 2.6387
Epoch 6/10
19/19 [==============================] - 0s - loss: 45.1753 - mean_squared_error: 5.3576
Epoch 7/10
19/19 [==============================] - 0s - loss: 45.1275 - mean_squared_error: 5.4549
Epoch 8/10
19/19 [==============================] - 0s - loss: 44.4413 - mean_squared_error: 5.2160
Epoch 9/10
19/19 [==============================] - 0s - loss: 63.6388 - mean_squared_error: 7.5040
Epoch 10/10
19/19 [==============================] - 0s - loss: 61.3393 - mean_squared_error: 7.6078


In [28]:
X_val, y_val, w = next(image_label_generator(PATH_TO_DATA))

In [29]:
model.predict(X_val)

array([[  8.69347763],
       [ 47.21813202],
       [  8.31283951],
       [  5.42915535],
       [ 12.9414835 ],
       [ 12.13756371],
       [ 41.34660721],
       [ 11.77419376],
       [ 11.03258514],
       [ 49.16252518],
       [  9.37642574],
       [  8.07772827],
       [  8.46710205],
       [  8.12908363],
       [ 11.30315876],
       [ 11.1290493 ],
       [ 44.84928513],
       [  5.59077358],
       [ 13.10061836]], dtype=float32)

In [30]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [31]:
y_val

array([-0.04076847, -0.1167233 ,  0.07132844,  0.        ,  0.07132844,
       -0.05975719,  0.        ,  0.        ,  0.1670138 , -0.09773462,
       -0.05975719,  0.0617599 ,  0.0617599 ,  0.1670138 ,  0.1670138 ,
        0.01391724,  0.        ,  0.0617599 ,  0.07132844])